In [ ]:
# !pip install tensorflow==1.15.0

In [ ]:
# !pip install tensorflow-gpu==1.15.0

In [2]:
import sys
sys.path.append("..")
import matplotlib.pyplot as plt
from scziDesk_preprocess import *
from scziDesk_network import *
from scziDesk_utils import *
import argparse
import h5py
import time
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, silhouette_score, calinski_harabasz_score
from collections import Counter
import glob2
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

/opt/conda/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,
Using TensorFlow backend.


In [3]:
tf.__version__

'1.15.0'

In [4]:
random_seed = [1111, 2222, 3333, 4444, 5555, 6666, 7777, 8888, 9999, 10000]

parser = argparse.ArgumentParser(description="train", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
# parser.add_argument("--dataname", default = "Quake_10x_Bladder", type = str)
parser.add_argument("--dataname", default = "data_-1.5c15", type = str)

parser.add_argument("--distribution", default = "ZINB")
parser.add_argument("--self_training", default = True)
parser.add_argument("--dims", default = [499, 256, 64, 32])
parser.add_argument("--highly_genes", default = 500)
parser.add_argument("--alpha", default = 0.001, type = float)
parser.add_argument("--gamma", default = 0.001, type = float)
parser.add_argument("--learning_rate", default = 0.0001, type = float)
parser.add_argument("--random_seed", default = random_seed)
parser.add_argument("--batch_size", default = 256, type = int)
parser.add_argument("--update_epoch", default = 10, type = int)
parser.add_argument("--pretrain_epoch", default = 1000, type = int)
parser.add_argument("--funetrain_epoch", default = 2000, type = int)
parser.add_argument("--t_alpha", default = 1.0)
parser.add_argument("--noise_sd", default = 1.5)
parser.add_argument("--error", default = 0.001, type = float)
parser.add_argument("--gpu_option", default = "0")

args = parser.parse_args("")

In [ ]:
for category in ["balanced_data", "imbalanced_data", "real_data"
                ]:

    path= ".."
    if category in ["balanced_data", "imbalanced_data"]:
        files = glob2.glob(f'{path}/R/simulated_data/{category}/*.h5')
        files = [f[len(f"{path}/R/simulated_data/{category}/"):-3] for f in files]
    else:
        files = glob2.glob(f'{path}/real_data/*.h5')
        files = [f[len(f"{path}/real_data/"):-3] for f in files]
    print(files)

    df = pd.DataFrame(columns = ["dataset", "ARI", "NMI", "sil", "run", "time", "pred", "cal"])
    for dataset in files:
        if category in ["balanced_data", "imbalanced_data"]:
            data_mat = h5py.File(f"{path}/R/simulated_data/{category}/{dataset}.h5","r")
        else:
            data_mat = h5py.File(f"{path}/real_data/{dataset}.h5","r")

        Y = np.array(data_mat['Y'])
        X = np.array(data_mat['X'])
        print(f">>>>dataset {dataset}")
        if X.shape[0] > 10000:
            continue

        X = np.ceil(X).astype(np.int)
        count_X = X
        print(X.shape, count_X.shape)
        orig_X = X.copy()
        adata = sc.AnnData(X)
        adata.obs['Group'] = Y
        adata = normalize(adata,
                          copy=True,
                          highly_genes=args.highly_genes,
                          size_factors=True,
                          normalize_input=True,
                          logtrans_input=True)
        X = adata.X.astype(np.float32)
        Y = np.array(adata.obs["Group"])

        high_variable = np.array(adata.var.highly_variable.index, dtype=np.int)
        count_X = count_X[:, high_variable]
        size_factor = np.array(adata.obs.size_factors).reshape(-1,
                                                               1).astype(np.float32)
        cluster_number = int(max(Y) - min(Y) + 1)
        args.dims[0]=X.shape[1]
        print(X.shape, count_X.shape)

        for run in range(3):
            start = time.time()
            seed = run
            np.random.seed(seed)
            tf.reset_default_graph()
            chencluster = autoencoder(args.dataname, args.distribution,
                                      args.self_training, args.dims, cluster_number,
                                      args.t_alpha, args.alpha, args.gamma,
                                      args.learning_rate, args.noise_sd)
            chencluster.pretrain(X, count_X, size_factor, args.batch_size,
                                 args.pretrain_epoch, args.gpu_option)

            chencluster.funetrain(X, count_X, size_factor, args.batch_size,
                                  args.funetrain_epoch, args.update_epoch, args.error, Y)

            kmeans_ARI = np.around(adjusted_rand_score(Y, chencluster.kmeans_pred), 5)
            kmeans_NMI = np.around(
                normalized_mutual_info_score(Y, chencluster.kmeans_pred), 5)

            elapsed = time.time() - start
            ARI = adjusted_rand_score(Y, chencluster.Y_pred)
            NMI = np.around(normalized_mutual_info_score(Y, chencluster.Y_pred), 5)
            ss = silhouette_score(chencluster.latent_repre,chencluster.Y_pred)
            cal = calinski_harabasz_score(chencluster.latent_repre,chencluster.Y_pred)

            df.loc[df.shape[0]] = [dataset, ARI, NMI, ss, run, elapsed, chencluster.Y_pred, cal]
            df.to_pickle(f"../output/pickle_results/{category}/{category}_sczi.pkl")


['data_1c8', 'data_-1c4', 'data_-1c8', 'data_0c4', 'data_0c8', 'data_0c16', 'data_1.5c4', 'data_1c4', 'data_1.5c8', 'data_1.5c16', 'data_-1c16', 'data_1c16']
>>>>dataset data_1c8
(2000, 2500) (2000, 2500)
(2000, 499) (2000, 499)
begin the pretraining
begin the funetraining
ARI 0.35907, NMI 0.4231
0 ARI 0.35907, NMI 0.4231
begin the pretraining
begin the funetraining
ARI 0.41119, NMI 0.45917
0 ARI 0.41119, NMI 0.45917
begin the pretraining
begin the funetraining
ARI 0.40315, NMI 0.48406
0 ARI 0.40348, NMI 0.4848
10 ARI 0.44017, NMI 0.51366
20 ARI 0.45471, NMI 0.52476
30 ARI 0.46285, NMI 0.53179
40 ARI 0.46608, NMI 0.53445
50 ARI 0.46767, NMI 0.5354
60 ARI 0.46811, NMI 0.53647
70 ARI 0.47052, NMI 0.53883
80 ARI 0.47318, NMI 0.54143
90 ARI 0.47318, NMI 0.54143
>>>>dataset data_-1c4
(1000, 2500) (1000, 2500)
(1000, 499) (1000, 499)
begin the pretraining
begin the funetraining
ARI 0.95403, NMI 0.92952
0 ARI 0.95403, NMI 0.92952
begin the pretraining
begin the funetraining
ARI 0.94537, NMI 0

1970 ARI 0.54399, NMI 0.62585
1980 ARI 0.54577, NMI 0.62544
1990 ARI 0.54421, NMI 0.62599
2000 ARI 0.54335, NMI 0.62344
begin the pretraining
begin the funetraining
ARI 0.40126, NMI 0.49064
0 ARI 0.40121, NMI 0.49042
>>>>dataset data_1.5c4
(1000, 2500) (1000, 2500)
(1000, 499) (1000, 499)
begin the pretraining
begin the funetraining
ARI 0.52655, NMI 0.50051
0 ARI 0.52655, NMI 0.50051
begin the pretraining
begin the funetraining
ARI 0.4937, NMI 0.49487
0 ARI 0.4937, NMI 0.49487
begin the pretraining
begin the funetraining
ARI 0.0603, NMI 0.07297
0 ARI 0.0603, NMI 0.07297
>>>>dataset data_1c4
(1000, 2500) (1000, 2500)
(1000, 499) (1000, 499)
begin the pretraining
begin the funetraining
ARI 0.88722, NMI 0.85913
0 ARI 0.88722, NMI 0.85913
begin the pretraining
begin the funetraining
ARI 0.48636, NMI 0.5182
0 ARI 0.48647, NMI 0.51826
10 ARI 0.4872, NMI 0.51698
20 ARI 0.49, NMI 0.51803
30 ARI 0.49098, NMI 0.51883
40 ARI 0.49172, NMI 0.51949
50 ARI 0.49487, NMI 0.52203
60 ARI 0.49616, NMI 0.5

130 ARI 0.03308, NMI 0.08255
140 ARI 0.03307, NMI 0.08263
150 ARI 0.03349, NMI 0.08341
160 ARI 0.03368, NMI 0.08371
170 ARI 0.03372, NMI 0.08385
180 ARI 0.03374, NMI 0.08366
190 ARI 0.03376, NMI 0.08376
200 ARI 0.0342, NMI 0.08456
210 ARI 0.03424, NMI 0.08461
220 ARI 0.03422, NMI 0.08466
230 ARI 0.0346, NMI 0.08543
240 ARI 0.03474, NMI 0.08568
250 ARI 0.03482, NMI 0.08587
260 ARI 0.03475, NMI 0.08565
270 ARI 0.03483, NMI 0.08582
280 ARI 0.03508, NMI 0.08618
290 ARI 0.03546, NMI 0.08688
300 ARI 0.03539, NMI 0.08712
310 ARI 0.03549, NMI 0.08763
320 ARI 0.03572, NMI 0.08829
330 ARI 0.0358, NMI 0.08842
340 ARI 0.03592, NMI 0.0888
350 ARI 0.0359, NMI 0.08869
360 ARI 0.036, NMI 0.08886
370 ARI 0.03624, NMI 0.08953
380 ARI 0.03621, NMI 0.08963
390 ARI 0.03687, NMI 0.09075
400 ARI 0.03717, NMI 0.09138
410 ARI 0.03704, NMI 0.09122
420 ARI 0.03703, NMI 0.09135
430 ARI 0.03737, NMI 0.09205
440 ARI 0.03755, NMI 0.09241
450 ARI 0.038, NMI 0.09316
460 ARI 0.03852, NMI 0.09386
470 ARI 0.03867, NMI 0.

910 ARI 0.07456, NMI 0.13152
920 ARI 0.07447, NMI 0.1315
930 ARI 0.07461, NMI 0.13226
940 ARI 0.07532, NMI 0.13264
950 ARI 0.07638, NMI 0.13329
960 ARI 0.07529, NMI 0.13274
970 ARI 0.07498, NMI 0.13256
980 ARI 0.07599, NMI 0.13372
990 ARI 0.07727, NMI 0.13573
1000 ARI 0.07752, NMI 0.13644
1010 ARI 0.07607, NMI 0.13453
1020 ARI 0.07722, NMI 0.13542
1030 ARI 0.07666, NMI 0.13482
1040 ARI 0.07642, NMI 0.13542
1050 ARI 0.07701, NMI 0.13581
1060 ARI 0.07735, NMI 0.13582
1070 ARI 0.07702, NMI 0.13556
1080 ARI 0.07748, NMI 0.13656
1090 ARI 0.07816, NMI 0.13754
1100 ARI 0.07794, NMI 0.13656
1110 ARI 0.078, NMI 0.13668
1120 ARI 0.07701, NMI 0.13587
1130 ARI 0.07838, NMI 0.13742
1140 ARI 0.07838, NMI 0.13688
1150 ARI 0.07763, NMI 0.13612
1160 ARI 0.07907, NMI 0.13791
1170 ARI 0.07964, NMI 0.13846
1180 ARI 0.0803, NMI 0.13975
1190 ARI 0.07942, NMI 0.13827
1200 ARI 0.08025, NMI 0.1391
1210 ARI 0.08065, NMI 0.13906
1220 ARI 0.08012, NMI 0.13856
1230 ARI 0.08021, NMI 0.13898
1240 ARI 0.08079, NMI 0.

1650 ARI 0.83519, NMI 0.84464
1660 ARI 0.83869, NMI 0.8473
1670 ARI 0.83509, NMI 0.84426
1680 ARI 0.8378, NMI 0.84652
1690 ARI 0.84139, NMI 0.85023
1700 ARI 0.84217, NMI 0.85066
1710 ARI 0.83584, NMI 0.8457
1720 ARI 0.83897, NMI 0.8479
1730 ARI 0.83988, NMI 0.84869
1740 ARI 0.83978, NMI 0.84858
1750 ARI 0.84372, NMI 0.8511
1760 ARI 0.83607, NMI 0.8447
1770 ARI 0.84087, NMI 0.84868
1780 ARI 0.83833, NMI 0.8461
1790 ARI 0.83636, NMI 0.84484
1800 ARI 0.83934, NMI 0.8479
1810 ARI 0.83915, NMI 0.84745
1820 ARI 0.84256, NMI 0.85119
1830 ARI 0.84104, NMI 0.85025
1840 ARI 0.84007, NMI 0.84928
1850 ARI 0.84299, NMI 0.85199
1860 ARI 0.84417, NMI 0.8528
1870 ARI 0.84254, NMI 0.85132
1880 ARI 0.84179, NMI 0.84997
1890 ARI 0.84268, NMI 0.85082
1900 ARI 0.83986, NMI 0.84861
1910 ARI 0.83749, NMI 0.84633
1920 ARI 0.84109, NMI 0.84932
1930 ARI 0.8421, NMI 0.85051
1940 ARI 0.839, NMI 0.84764
1950 ARI 0.83861, NMI 0.84792
1960 ARI 0.83968, NMI 0.84824
1970 ARI 0.83543, NMI 0.8446
1980 ARI 0.84097, NMI 0

...

1500 ARI 0.33393, NMI 0.36671
1510 ARI 0.32948, NMI 0.36191
1520 ARI 0.332, NMI 0.364
1530 ARI 0.33238, NMI 0.36489
1540 ARI 0.32953, NMI 0.36098
1550 ARI 0.32927, NMI 0.36241
1560 ARI 0.32924, NMI 0.36083
1570 ARI 0.33054, NMI 0.3635
1580 ARI 0.33212, NMI 0.36355
1590 ARI 0.33116, NMI 0.36424
1600 ARI 0.3321, NMI 0.36358
1610 ARI 0.33368, NMI 0.3661
1620 ARI 0.33031, NMI 0.36199
1630 ARI 0.33031, NMI 0.36327
1640 ARI 0.3284, NMI 0.3609
1650 ARI 0.3319, NMI 0.3622
1660 ARI 0.32905, NMI 0.36105
1670 ARI 0.33047, NMI 0.36164
1680 ARI 0.3269, NMI 0.36087
1690 ARI 0.32904, NMI 0.35936
1700 ARI 0.32794, NMI 0.35973
1710 ARI 0.33034, NMI 0.36054
1720 ARI 0.32717, NMI 0.36013
1730 ARI 0.3289, NMI 0.35949
1740 ARI 0.32787, NMI 0.35865
1750 ARI 0.32872, NMI 0.36048
1760 ARI 0.32795, NMI 0.35822
1770 ARI 0.32604, NMI 0.35707
1780 ARI 0.32807, NMI 0.35728
1790 ARI 0.32503, NMI 0.35646
1800 ARI 0.32903, NMI 0.35839
1810 ARI 0.32521, NMI 0.35598
1820 ARI 0.32801, NMI 0.35767
1830 ARI 0.33111, NMI 0

130 ARI 0.48226, NMI 0.56226
140 ARI 0.48295, NMI 0.56229
150 ARI 0.48299, NMI 0.56135
160 ARI 0.4836, NMI 0.56253
170 ARI 0.48435, NMI 0.56341
180 ARI 0.48513, NMI 0.56343
190 ARI 0.48577, NMI 0.56434
200 ARI 0.48595, NMI 0.5648
210 ARI 0.48681, NMI 0.56552
220 ARI 0.48705, NMI 0.56628
230 ARI 0.4861, NMI 0.56582
240 ARI 0.48741, NMI 0.56745
250 ARI 0.48737, NMI 0.56759
260 ARI 0.48615, NMI 0.56577
270 ARI 0.48692, NMI 0.56739
280 ARI 0.48735, NMI 0.56732
290 ARI 0.4866, NMI 0.56687
300 ARI 0.48792, NMI 0.56759
310 ARI 0.48644, NMI 0.56617
320 ARI 0.48747, NMI 0.56745
330 ARI 0.48858, NMI 0.56837
340 ARI 0.48884, NMI 0.56806
350 ARI 0.48903, NMI 0.5676
360 ARI 0.48873, NMI 0.56755
begin the pretraining
begin the funetraining
ARI 0.51136, NMI 0.59261
0 ARI 0.51116, NMI 0.59278
10 ARI 0.49469, NMI 0.57934
20 ARI 0.49212, NMI 0.57839
30 ARI 0.49172, NMI 0.58021
40 ARI 0.49275, NMI 0.58318
50 ARI 0.49321, NMI 0.58373
60 ARI 0.4955, NMI 0.58584
70 ARI 0.49635, NMI 0.58638
80 ARI 0.49627, N

100 ARI 0.40101, NMI 0.43281
110 ARI 0.40056, NMI 0.43313
120 ARI 0.40176, NMI 0.43481
130 ARI 0.40276, NMI 0.4368
140 ARI 0.40208, NMI 0.43705
150 ARI 0.40393, NMI 0.43921
160 ARI 0.40408, NMI 0.44105
170 ARI 0.40427, NMI 0.44069
180 ARI 0.40603, NMI 0.44564
190 ARI 0.40753, NMI 0.44752
200 ARI 0.40734, NMI 0.44746
>>>>dataset data_0c8
(3000, 2500) (3000, 2500)
(3000, 499) (3000, 499)
begin the pretraining
begin the funetraining
ARI 0.32433, NMI 0.36344
0 ARI 0.32465, NMI 0.36379
10 ARI 0.32544, NMI 0.36416
20 ARI 0.32834, NMI 0.36754
30 ARI 0.32709, NMI 0.36548
40 ARI 0.33015, NMI 0.36796
50 ARI 0.33189, NMI 0.37017
60 ARI 0.33364, NMI 0.3721
70 ARI 0.33302, NMI 0.37136
80 ARI 0.33314, NMI 0.37181
90 ARI 0.33548, NMI 0.3743
100 ARI 0.33635, NMI 0.37529
110 ARI 0.33757, NMI 0.37628
120 ARI 0.33759, NMI 0.37638
130 ARI 0.33982, NMI 0.37887
140 ARI 0.34027, NMI 0.37962
150 ARI 0.34223, NMI 0.38151
160 ARI 0.3429, NMI 0.38202
170 ARI 0.34297, NMI 0.38207
180 ARI 0.34271, NMI 0.38213
190 

600 ARI 0.36838, NMI 0.39193
610 ARI 0.37202, NMI 0.39369
620 ARI 0.37217, NMI 0.39488
630 ARI 0.37459, NMI 0.39691
640 ARI 0.37679, NMI 0.39821
650 ARI 0.37607, NMI 0.39811
660 ARI 0.38132, NMI 0.4029
670 ARI 0.37889, NMI 0.39998
680 ARI 0.38126, NMI 0.40225
690 ARI 0.38205, NMI 0.40375
700 ARI 0.38439, NMI 0.40505
710 ARI 0.38248, NMI 0.40426
720 ARI 0.38473, NMI 0.40657
730 ARI 0.38421, NMI 0.40562
740 ARI 0.38621, NMI 0.40765
750 ARI 0.38437, NMI 0.40591
760 ARI 0.38563, NMI 0.40734
770 ARI 0.38689, NMI 0.4074
780 ARI 0.3861, NMI 0.40711
790 ARI 0.38936, NMI 0.40978
800 ARI 0.3889, NMI 0.40902
810 ARI 0.39137, NMI 0.41169
820 ARI 0.38869, NMI 0.40952
830 ARI 0.39052, NMI 0.4112
840 ARI 0.39081, NMI 0.41131
850 ARI 0.39153, NMI 0.41159
860 ARI 0.39073, NMI 0.4116
870 ARI 0.39012, NMI 0.41048
880 ARI 0.39204, NMI 0.41249
890 ARI 0.39083, NMI 0.41106
900 ARI 0.39026, NMI 0.41034
910 ARI 0.38998, NMI 0.41148
920 ARI 0.38946, NMI 0.40954
930 ARI 0.39003, NMI 0.41105
940 ARI 0.39089, NMI

1100 ARI 0.12843, NMI 0.17277
1110 ARI 0.12924, NMI 0.17288
1120 ARI 0.1299, NMI 0.17399
1130 ARI 0.12998, NMI 0.17303
1140 ARI 0.13027, NMI 0.17409
1150 ARI 0.13045, NMI 0.17398
1160 ARI 0.13206, NMI 0.17523
1170 ARI 0.13212, NMI 0.17508
1180 ARI 0.13279, NMI 0.17631
1190 ARI 0.1319, NMI 0.1757
1200 ARI 0.13494, NMI 0.17729
1210 ARI 0.13381, NMI 0.17776
1220 ARI 0.13458, NMI 0.17799
1230 ARI 0.13606, NMI 0.17887
1240 ARI 0.13776, NMI 0.18029
1250 ARI 0.13618, NMI 0.17914
1260 ARI 0.1372, NMI 0.17914
1270 ARI 0.13723, NMI 0.17974
1280 ARI 0.13812, NMI 0.18095
1290 ARI 0.13708, NMI 0.17954
1300 ARI 0.13837, NMI 0.18061
1310 ARI 0.13695, NMI 0.18053
1320 ARI 0.13771, NMI 0.18054
1330 ARI 0.13924, NMI 0.18106
1340 ARI 0.13844, NMI 0.18099
1350 ARI 0.13859, NMI 0.1824
1360 ARI 0.13918, NMI 0.18164
1370 ARI 0.1384, NMI 0.18178
1380 ARI 0.13933, NMI 0.18246
1390 ARI 0.14008, NMI 0.18426
1400 ARI 0.13919, NMI 0.1816
1410 ARI 0.14012, NMI 0.18345
1420 ARI 0.13985, NMI 0.18319
1430 ARI 0.14108,

In [6]:
df.groupby("dataset").mean()

,ARI,NMI,sil,time
dataset,,,,
10X_PBMC,0.584565,0.706333,0.064692,460.356967
10X_PBMC_select_2100,0.597950,0.706143,0.026049,100.385072
Adam,0.863456,0.857260,0.021948,174.062563
Muraro,0.724627,0.777293,0.047724,142.597028
Quake_10x_Bladder,0.983096,0.966417,0.119126,123.532811
Quake_10x_Limb_Muscle,0.961056,0.933257,0.026476,249.786629
Quake_10x_Spleen,0.909535,0.843680,0.209160,466.575191
Quake_Smart-seq2_Diaphragm,0.959670,0.935230,0.063294,49.739813
Quake_Smart-seq2_Limb_Muscle,0.973676,0.951237,0.078811,54.004468
